In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train_V2.csv')
test = pd.read_csv('../input/test_V2.csv')
train.head(4)
test.head(4)

In [ ]:
train.isnull().sum()

In [ ]:
columns = train.columns
columns

In [ ]:
train.dtypes

In [ ]:
train['matchType'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(train.corr())

In [ ]:
dummies = pd.get_dummies(train['matchType'])
train = pd.concat([train,dummies], axis=1)
train = train.drop("matchType",axis=1)


In [ ]:
dumm = pd.get_dummies(test['matchType'])
test = pd.concat([test,dumm], axis=1)
test = test.drop("matchType",axis=1)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
columns = train.columns
columns

In [ ]:
columns = columns.drop(['Id','groupId','matchId','winPlacePerc'])
columns

In [ ]:
columns.shape

In [ ]:
corr = train.corr()
corr

In [ ]:
sorted_cors = corr['winPlacePerc'].abs().sort_values()

In [ ]:
sorted_cors

def plot_correlation_heatmap(df):
    corr = df.corr()
    
    sns.set(style="white")
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)


    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()


plot_correlation_heatmap(train[columns])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220, 10, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))


sns.heatmap(corr,mask=mask,cmap=cmap,vmax=.3,center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
strong_corrs = sorted_cors[sorted_cors>0.20]
corrmat = train[strong_corrs.index].corr()
sns.heatmap(corrmat)

In [ ]:
corrmat.isnull().sum()

In [ ]:
cols = train[strong_corrs.index]
cols

In [ ]:
column = cols.columns
column

In [ ]:
train[column] = train[column].fillna(train[column].mean())


In [ ]:
column = column.drop('winPlacePerc')

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression


all_X = train[column]
all_y = train["winPlacePerc"]

lr = LinearRegression()
selector = RFECV(lr,cv=10)
selector.fit(all_X,all_y)
optimized_columns = all_X.columns[selector.support_]

In [ ]:
optimized_columns

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr,all_X,all_y,cv=10)
accuracy = scores.mean()
accuracy

In [ ]:
test[column].isnull().sum()

In [ ]:
lr.fit(all_X,all_y)
prediction = lr.predict(test[column])
prediction

In [ ]:
holdout_id = test['Id']
submission_df = {'Id':holdout_id,
                 'winPlacePerc':prediction}
submission = pd.DataFrame(submission_df)
submission.to_csv('lr.csv', index=False)